# Team Info

**IS4242 Project**

Driven Data Team Name: **Zup cai peng**

Team Members:

*   Chen Ri Sheng (A0182448X)
*   Phang Tze Ming (A0184247Y)




# Part 2: Prescriptive Analytics

In [1]:
pip install pyomo

You should consider upgrading via the '/Users/p.tm/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyomo.environ as pe
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians

## Preprocessing

We are combining our training and test data with our predicted y labels using our best model from Part 1.

In [3]:
x_train = pd.read_csv('x_train.csv')
x_test = pd.read_csv('x_test.csv')
y_train = pd.read_csv('y_train.csv')

# for y_pred_best, we are using the results from part 1 of the project, where we have obtained the score of 0.7830
y_pred_best = pd.read_csv('y_pred_best.csv')

In [4]:
# left join the dataframes for train set
trainset = pd.merge(x_train, y_train, left_on='id', right_on='id', how='left')
# trainset

In [5]:
# left join the dataframes for test set
testset = pd.merge(x_test, y_pred_best, left_on='id', right_on='id', how='left')
# testset

In [6]:
# combining test set and train set into 1 dataframe
totalset = pd.concat([trainset, testset], ignore_index=True)
totalset.shape

(74250, 41)

Here we extract the necessary features for our optimization model.

In [7]:
# Remove unecessary features that we are not considering for this analytics
dataset = totalset[['status_group', 'amount_tsh', 'lga', 'population', 'longitude', 'latitude', 'wpt_name']]
dataset.shape

(74250, 7)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74250 entries, 0 to 74249
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   status_group  74250 non-null  object 
 1   amount_tsh    74250 non-null  float64
 2   lga           74250 non-null  object 
 3   population    74250 non-null  int64  
 4   longitude     74250 non-null  float64
 5   latitude      74250 non-null  float64
 6   wpt_name      74250 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 4.0+ MB


Here we drop the rows that have no location information, no name, no surrounding populations or pumps that need repair. 

In [9]:
# based on the assumptions and the preprocessing steps in the project slides, we are getting the needed rows with the relevant column values
dataset = dataset[dataset.status_group != 'functional needs repair']
dataset = dataset[dataset.population > 0]
dataset = dataset[dataset.wpt_name != 'none']
dataset = dataset[dataset.lga != 'none']
dataset

,status_group,amount_tsh,lga,population,longitude,latitude,wpt_name
1,functional,0.0,Serengeti,280,34.698766,-2.147466,Zahanati
2,functional,25.0,Simanjiro,250,37.460664,-3.821329,Kwa Mahundi
3,non functional,0.0,Nanyumbu,58,38.486161,-11.155298,Zahanati Ya Nanyumbu
5,functional,20.0,Mkinga,1,39.172796,-4.765587,Tajiri
10,functional,0.0,Mkuranga,345,39.209518,-7.034139,Mzee Hokororo
...,...,...,...,...,...,...,...
74243,non functional,0.0,Kigoma Rural,1400,29.768139,-4.480618,Mwandami
74245,functional,0.0,Bagamoyo,20,38.852669,-6.582841,Kwambwezi
74246,functional,1000.0,Kilindi,2960,37.451633,-5.350428,Bonde La Mkondoa
74248,functional,0.0,Songea Rural,150,35.432732,-10.584159,Kwa John


## Constructing the Model

Here we seperate the functional pumps from the non functional.

In [10]:
# separate functional and non-functional pumps
functional = dataset[dataset.status_group != 'non functional']
non_functional = dataset[dataset.status_group != 'functional']

print('Functional shape: ', functional.shape)
print('Non-functional shape: ', non_functional.shape)

non_functional = non_functional[non_functional.lga == 'Nanyumbu']
non_functional.shape

Functional shape:  (24987, 7)
Non-functional shape:  (16912, 7)


(155, 7)

Our objective here is to minimize the total cost to serve water to everyone. To do that, we identified 2 variables that directly affect the cost:
+ A **binary variable** that determines the pump should be replaced: $x_i$, where `i` refers to non-functioning pump `i`
+ A **binary variable** that determines the pump should have water transported from the nearest functioning pump: $y_i$, where `i` refers to non-functioning pump `i`

To minimize the total cost:
$$\sum_{i=1}^n x_i (100 + 0.05 * P_i.TSH) + y_i(100 + 2000 * d)$$
Where 
+ `n` : total number of non-functioning pumps
+ $P_i.TSH$ : `amount_tsh` of non-functioning pump i
+ `d` : Euclidean distance between $P_i$ and its nearest functioning pump

Next we define a method that will construct a ConcreteModel based on the LGA specified by the client. For the model, we have identified the following constraints:

+ *Pump Constraint* : Non-functioning pumps must either be replaced or draw water from its nearest function pump
+ *Population Constraint* : The sum of the population of non-functioning pumps that is drawing water from a functioning pump must be less that it's population (assuming functioning pumps can serve twice as many as its surrounding population) 

We translated these constraints into the following:
+ Pump Constraint: $\quad x_i + y_i == 1 \quad$ for each non-functioning pump i = 1 to n 
+ Population Constraint: $\quad \sum_{i=1}^m P_i.population < z_j \quad$ for each functioning pump $z$ and `m` non-functioning pumps drawing water from $z$

In this method, we also defined the following helper methods:
+ `get_euclidean_dist` : takes in 2 pumps as parameters and returns the euclidean distance
+ `get_nearest_pump` : takes in a non-functioning pump and returns the nearest functioning pump that has a higher population

In [11]:
def create_model(LGA): 
    # instantiate our model
    pump = pe.ConcreteModel(name=f'Replace or not: {LGA}')

    # get the functional and non-functional pumps within the LGA
    lga_functional = functional[functional.lga == LGA]
    lga_non_functional = non_functional[non_functional.lga == LGA]
    
    # converting the pump lists to indices
    func_id = lga_functional.index.tolist()
    non_func_id = lga_non_functional.index.tolist()
    
    # declaring the variables
    pump.replace = pe.Var(non_func_id, domain=pe.Binary, initialize=0)
    pump.transport = pe.Var(non_func_id, domain=pe.Binary, initialize=0)
    
    # helper method
    def get_euc_dist(non_func_pump, func_pump):
      lat_diff = non_func_pump.latitude - func_pump.latitude
      lng_diff = non_func_pump.longitude - func_pump.longitude

      return sqrt(pow(lat_diff, 2) + pow(lng_diff, 2))

    # helper method
    def get_nearest_pump(pump):
      nearest = functional.iloc[0]
      nearest_dist = get_euc_dist(pump, lga_functional.iloc[0])

      for id in lga_functional.index.tolist():
        func_pump = lga_functional.loc[id]
        dist = get_euc_dist(pump, func_pump)
        if dist < nearest_dist and pump.population <= func_pump.population:
          nearest_dist = dist
          nearest = lga_functional.loc[id]  
      return nearest
    
    # Pump constraint
    def pumpcon_rule(model, i):
      return model.replace[i] + model.transport[i] == 1
    pump.pumpcon = pe.Constraint(non_func_id, rule=pumpcon_rule)
    
    # Population constraint
    def popcon_rule(model, i):
        population = []
        for id in non_func_id:
            curr_pump = non_functional.loc[id]
            if model.transport[id] != 0 and nearest_pump_mapping[id] == i:
                population.append(curr_pump.population)
        # end for
        return (0, sum(population), functional.loc[i].population)
    pump.popcon = pe.Constraint(func_id, rule=popcon_rule)
    
    # Objective to minimize
    def obj_rule(model):
      costs = 0
      for id in non_func_id:
        pump = lga_non_functional.loc[id]
        costs += model.replace[id] * (100 + 0.05 * pump.amount_tsh)
        costs += model.transport[id] * (100 + 2000 * get_euc_dist(pump, get_nearest_pump(pump)))
      return costs

    pump.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)
    
    return pump

Next, we define a method that takes in the LGA and returns the list of pumps that needs to be replaced as well as the total cost of replacing these pumps or transporting water from the nearest available functioning pump. 

For this, we assumed that the non-functioning pumps within a specified LGA can only transport water from functioning pumps within the same LGA. **We did not consider functioning pumps outside of the specified LGA.**

In [12]:
def get_pumps_and_cost(LGA):
    instance = create_model(LGA)
    pe.SolverFactory('glpk').solve(instance)
    
    pumps = []
    for id in instance.replace:
        if instance.replace[id].value == 1:
            pump = non_functional.loc[id]
            pumps.append(pump.wpt_name)
    
    return pumps, instance.obj.expr()
    

The following is the sample usage and output of our model.

In [13]:
pumps, total_cost = get_pumps_and_cost('Nanyumbu')
print(pd.DataFrame(pumps, columns=['Pumps to replace']))
print('Total cost: ${:0.2f}'.format(total_cost))

         Pumps to replace
0    Zahanati Ya Nanyumbu
1       Kwa Zena Rajabu 2
2                Namajani
3               Kwa Peter
4                 Mission
..                    ...
147              Mianzini
148  Kwa Juma Idi Kambutu
149        Kwa Amiri Abed
150     Kwa Mwenye Nipoto
151              Mkwajuni

[152 rows x 1 columns]
Total cost: $15527.59
